In [1]:
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from keras.preprocessing import image
from keras.applications.resnet import ResNet101, preprocess_input
from tqdm import tqdm
import torch
import pandas as pd
from sklearn.cluster import KMeans
import shutil

# Define paths
input_dir = '/kaggle/input/lfw-dataset/lfw-deepfunneled/lfw-deepfunneled'
weights_path = '/kaggle/input/resnet101-trained-model-faces-30/resnet101_trained_model_faces_30.pth'
output_dir = '/kaggle/working/'

# Load model with PyTorch weights (Keras model)
model = ResNet101(weights='imagenet', include_top=False)
if os.path.exists(weights_path):
    state_dict = torch.load(weights_path, map_location=torch.device('cpu'))
    for layer in model.layers:
        if layer.name in state_dict:
            layer.set_weights(state_dict[layer.name])

def extract_features(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array).flatten()
    # L2 normalize the feature vector
    features = features / np.linalg.norm(features)
    return features

def create_ground_truth_matrix(image_paths):
    """Create binary ground truth matrix where 1 = same identity, 0 = different."""
    identities = [os.path.dirname(p) for p in image_paths]
    return np.equal.outer(identities, identities).astype(int)

def apply_paper_transformation(sim_matrix, tau=0.7):
    """Apply piecewise similarity transformation from the paper."""
    transformed = np.where(sim_matrix >= tau, sim_matrix, sim_matrix/(2*tau))
    return np.clip(transformed, 0, 1)

def inject_ground_truth(similarity_matrix, ground_truth_matrix, injection_ratio=0.01):
    """Inject ground truth values into random columns of similarity matrix."""
    n = similarity_matrix.shape[0]
    cols_to_replace = np.random.choice(n, size=int(n * injection_ratio), replace=False)
    for col in cols_to_replace:
        similarity_matrix[:, col] = ground_truth_matrix[:, col]
        similarity_matrix[col, :] = ground_truth_matrix[col, :]
    return similarity_matrix

def create_adjacency_matrix(image_paths, tau=0.7):
    image_features = []
    for path in tqdm(image_paths, desc="Extracting features"):
        image_features.append(extract_features(path))
    
    similarities = cosine_similarity(image_features)
    ground_truth = create_ground_truth_matrix(image_paths)
    
    # Inject a fraction of ground truth columns and transform
    modified_similarities = inject_ground_truth(similarities, ground_truth)
    modified_similarities = apply_paper_transformation(modified_similarities, tau)
    
    return modified_similarities, ground_truth

class UnionFind:
    def __init__(self, size):
        self.parent = list(range(size))
    
    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]
    
    def union(self, x, y):
        fx = self.find(x)
        fy = self.find(y)
        if fx != fy:
            self.parent[fy] = fx

def conpac_clustering(similarity_matrix, max_iterations=10):
    def compute_unary_potentials(sim_matrix):
        sim_matrix = np.clip(sim_matrix, 1e-10, 1 - 1e-10)
        unary = np.zeros((*sim_matrix.shape, 2))
        unary[:, :, 1] = -np.log(sim_matrix)
        unary[:, :, 0] = -np.log(1 - sim_matrix)
        return unary

    def triplet_potential(y_ij, y_ik, y_jk):
        return (1 - y_ij) * y_ik * y_jk + y_ij * (1 - y_ik) * y_jk + y_ij * y_ik * (1 - y_jk)

    def propagate_messages(unary, adj_matrix, num_iter, tol=1e-3):
        n = adj_matrix.shape[0]
        messages = np.copy(unary)
        for t in range(num_iter):
            new_msgs = np.copy(messages)
            for i in range(n):
                for j in range(i + 1, n):
                    if adj_matrix[i, j] == 0:
                        continue
                    active_neighbors = []
                    for k in range(n):
                        if k == i or k == j:
                            continue
                        if (abs(messages[i, k, 0] - messages[i, k, 1]) > tol or
                                abs(messages[j, k, 0] - messages[j, k, 1]) > tol):
                            active_neighbors.append(k)
                    if len(active_neighbors) == 0:
                        continue
                    for state in [0, 1]:
                        total = 0.0
                        for k in active_neighbors:
                            local_min = np.inf
                            for y_ik in [0, 1]:
                                for y_jk in [0, 1]:
                                    if triplet_potential(state, y_ik, y_jk) == 0:
                                        energy = messages[i, k, y_ik] + messages[j, k, y_jk]
                                        if energy < local_min:
                                            local_min = energy
                            total += local_min
                        new_msgs[i, j, state] = unary[i, j, state] + total
                    min_val = np.min(new_msgs[i, j, :])
                    new_msgs[i, j, :] -= min_val
                    new_msgs[j, i, :] = new_msgs[i, j, :]
            messages = new_msgs
        return messages

    # Normalize and transform similarity matrix
    sim_matrix = (similarity_matrix - similarity_matrix.min()) / (similarity_matrix.max() - similarity_matrix.min())
    np.fill_diagonal(sim_matrix, 1.0)

    unary = compute_unary_potentials(sim_matrix)
    adj_matrix = (sim_matrix >= 0.5).astype(int)

    messages = propagate_messages(unary, adj_matrix, max_iterations)

    # --------- Marginal belief matrix (confidence that i and j are in same cluster) ----------
    beliefs = np.exp(-messages)
    beliefs /= beliefs.sum(axis=2, keepdims=True)  # normalize
    marginal_belief_matrix = beliefs[:, :, 1]  # probability of being in the same cluster

    # --------- Final binary adjacency matrix based on message argmin ----------
    final_adj = np.zeros_like(adj_matrix)
    for i in range(final_adj.shape[0]):
        for j in range(i + 1, final_adj.shape[1]):
            final_adj[i, j] = 0 if messages[i, j, 0] < messages[i, j, 1] else 1
            final_adj[j, i] = final_adj[i, j]

    # --------- Final grouping using Union-Find ----------
    uf = UnionFind(final_adj.shape[0])
    for i in range(final_adj.shape[0]):
        for j in np.where(final_adj[i] == 1)[0]:
            if j > i:
                uf.union(i, j)

    final_adj = np.array([[int(uf.find(i) == uf.find(j))
                           for j in range(final_adj.shape[1])]
                          for i in range(final_adj.shape[0])])

    return final_adj, marginal_belief_matrix

def calculate_pairwise_metrics(ground_truth, predicted):
    y_true = ground_truth[np.triu_indices_from(ground_truth, k=1)]
    y_pred = predicted[np.triu_indices_from(predicted, k=1)]
    
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    fmeasure = 2 * (precision * recall) / (precision + recall + 1e-8)
    
    return precision, recall, fmeasure

def get_cluster_labels(adj_matrix):
    n = adj_matrix.shape[0]
    visited = [False] * n
    labels = [-1] * n
    current_label = 0
    for i in range(n):
        if not visited[i]:
            stack = [i]
            while stack:
                node = stack.pop()
                if not visited[node]:
                    visited[node] = True
                    labels[node] = current_label
                    neighbors = np.where(adj_matrix[node] == 1)[0]
                    for nb in neighbors:
                        if not visited[nb]:
                            stack.append(nb)
            current_label += 1
    return labels

# ==================== Main Execution ====================

# 1. Load 3000 image paths and shuffle the order
all_image_paths = [os.path.join(input_dir, root, file) 
                   for root, _, files in os.walk(input_dir) 
                   for file in files if file.endswith(('jpg', 'png'))][:3000]
np.random.shuffle(all_image_paths)

if all_image_paths:
    # 2. Compute the full similarity and ground truth matrices for 3000 images
    print("Computing full similarity matrix for 3000 images...")
    full_similarities, full_ground_truth = create_adjacency_matrix(all_image_paths)
    
    # 3. Start with 2500 images
    current_size = 2500
    sim_subset = full_similarities[:current_size, :current_size]
    gt_subset = full_ground_truth[:current_size, :current_size]
    
    print(f"Running clustering on initial set of {current_size} images...")
    adj_matrix_current, belief_matrix = conpac_clustering(sim_subset)
    precision, recall, fmeasure = calculate_pairwise_metrics(gt_subset, adj_matrix_current)
    print(f"Initial Metrics ({current_size} images) - Precision: {precision:.4f}, Recall: {recall:.4f}, F-measure: {fmeasure:.4f}")
    
    # 4. Incrementally add 100 images at a time until we reach 3000
    while current_size < 3000:
        next_size = min(current_size + 10, 3000)
        print(f"Adding {next_size - current_size} images (now {next_size} total)...")
        
        # Build merged matrix
        merged_sim_matrix = np.zeros((next_size, next_size))
        # Top-left: current clustering result
        merged_sim_matrix[:current_size, :current_size] = belief_matrix
        # Rest: original similarities from precomputed matrix
        merged_sim_matrix[:current_size, current_size:next_size] = full_similarities[:current_size, current_size:next_size]
        merged_sim_matrix[current_size:next_size, :current_size] = full_similarities[current_size:next_size, :current_size]
        merged_sim_matrix[current_size:next_size, current_size:next_size] = full_similarities[current_size:next_size, current_size:next_size]
        
        # Run clustering on merged matrix
        adj_matrix_merged, belief_matrix = conpac_clustering(merged_sim_matrix)
        
        # Calculate metrics for current merged set
        gt_merged = full_ground_truth[:next_size, :next_size]
        precision, recall, fmeasure = calculate_pairwise_metrics(gt_merged, adj_matrix_merged)
        print(f"Metrics ({next_size} images) - Precision: {precision:.4f}, Recall: {recall:.4f}, F-measure: {fmeasure:.4f}")
        
        current_size = next_size
    
    # 5. Save final results (now at 3000 images)
    final_adj_matrix = adj_matrix_merged
    pd.DataFrame(merged_sim_matrix).to_csv(os.path.join(output_dir, 'merged_similarities.csv'), index=False)
    pd.DataFrame(final_adj_matrix).to_csv(os.path.join(output_dir, 'adj_matrix_merged.csv'), index=False)
    
    # --- Create folders for each cluster under FINAL folder and copy images ---
    cluster_labels = get_cluster_labels(final_adj_matrix)
    final_folder = os.path.join(output_dir, "FINAL")
    if not os.path.exists(final_folder):
        os.makedirs(final_folder)
    
    for idx, img_path in enumerate(all_image_paths):
        cluster_folder = os.path.join(final_folder, f"cluster_{cluster_labels[idx]}")
        if not os.path.exists(cluster_folder):
            os.makedirs(cluster_folder)
        shutil.copy(img_path, cluster_folder)
    
    print("Images have been saved to their respective cluster folders under FINAL folder.")
else:
    print("No valid images found.")

  # ==================== SECOND FULL RUN ====================
print("\nRunning full clustering on all 3000 images from scratch...")
    
# Use the original 3000-image similarity and ground truth matrices
full_adj_matrix, full_belief_matrix = conpac_clustering(full_similarities)
full_precision, full_recall, full_fmeasure = calculate_pairwise_metrics(full_ground_truth, full_adj_matrix)
    
print(f"Full Run Metrics (3000 images) - Precision: {full_precision:.4f}, Recall: {full_recall:.4f}, F-measure: {full_fmeasure:.4f}")
    
    # Save results
pd.DataFrame(full_adj_matrix).to_csv(os.path.join(output_dir, 'adj_matrix_full_run.csv'), index=False)
    
    # Save clustered images from the full run
full_run_folder = os.path.join(output_dir, "FULL_RUN")
if not os.path.exists(full_run_folder):
    os.makedirs(full_run_folder)
    
full_cluster_labels = get_cluster_labels(full_adj_matrix)
for idx, img_path in enumerate(all_image_paths):
    cluster_folder = os.path.join(full_run_folder, f"cluster_{full_cluster_labels[idx]}")
    if not os.path.exists(cluster_folder):
        os.makedirs(cluster_folder)
    shutil.copy(img_path, cluster_folder)
print("Second full run completed. Images saved under FULL_RUN folder.")

171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


<ipython-input-1-f4ef06588799>:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(weights_path, map_location=torch.device('cpu'))


Computing full similarity matrix for 3000 images...


Extracting features:   0%|          | 0/3000 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


Extracting features: 100%|██████████| 3000/3000 [04:26<00:00, 11.26it/s]


Running clustering on initial set of 2500 images...


<ipython-input-1-f4ef06588799>:88: RuntimeWarning: divide by zero encountered in log
  unary[:, :, 0] = -np.log(1 - sim_matrix)


Initial Metrics (2500 images) - Precision: 0.6240, Recall: 0.7730, F-measure: 0.6905
Adding 10 images (now 2510 total)...
Metrics (2510 images) - Precision: 0.6523, Recall: 0.2354, F-measure: 0.3460
Adding 10 images (now 2520 total)...
Metrics (2520 images) - Precision: 0.6644, Recall: 0.2360, F-measure: 0.3482
Adding 10 images (now 2530 total)...
Metrics (2530 images) - Precision: 0.6617, Recall: 0.2300, F-measure: 0.3413
Adding 10 images (now 2540 total)...
Metrics (2540 images) - Precision: 0.6479, Recall: 0.2203, F-measure: 0.3288
Adding 10 images (now 2550 total)...
Metrics (2550 images) - Precision: 0.6350, Recall: 0.1999, F-measure: 0.3041
Adding 10 images (now 2560 total)...
Metrics (2560 images) - Precision: 0.6107, Recall: 0.1790, F-measure: 0.2768
Adding 10 images (now 2570 total)...
Metrics (2570 images) - Precision: 0.5957, Recall: 0.1669, F-measure: 0.2607
Adding 10 images (now 2580 total)...
Metrics (2580 images) - Precision: 0.5853, Recall: 0.1576, F-measure: 0.2484
Add